In [1]:
import os
#import openai# from dotenv import load_dotenv, find_dotenv
from dotenv import load_dotenv, find_dotenv


#from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatOllama
from langchain.llms import Ollama
from langchain.embeddings import OllamaEmbeddings

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

from langchain.vectorstores import DeepLake

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
ACTIVELOOP_TOKEN = os.environ['ACTIVELOOP_TOKEN']
ACTIVELOOP_USERNAME = os.environ['ACTIVELOOP_USERNAME']

#my_activeloop_org_id = "<YOUR-ACTIVELOOP-ORG-ID>"
my_activeloop_dataset_name = "langchain_course_indexers_retrievers"
dataset_path = f"hub://{ACTIVELOOP_USERNAME}/{my_activeloop_dataset_name}"


In [3]:
db_id = 'kb-material'# replace with your database name
#DeepLake.force_delete_by_path(f"hub://{ACTIVELOOP_USERNAME}/{db_id}")

In [4]:
# llm_model = "gpt-3.5-turbo"
# chat_open = ChatOpenAI(temperature=0.0, model=llm_model)

In [5]:
local_url = "http://localhost:11434"

In [6]:
llm = Ollama(base_url=local_url, model="mistral")

In [9]:
print(llm("Why is the sky blue?"))

 The reason why the sky appears blue during a clear day is due to a natural phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it interacts with different gases and particles in the air. Blue light has a shorter wavelength than other colors in the visible spectrum, making it more likely to collide with and be scattered by molecules of nitrogen and oxygen in the atmosphere.

As these blue light photons get scattered in all directions, they fill the sky with a scattering of blue hues, making the sky appear blue to our eyes when we look up. However, it's essential to note that the sky doesn't have a uniform blue color; instead, it varies from shades of baby blue near the horizon to a deeper, almost navy blue at the highest points in the sky. This happens due to how distance and air density affect the scattering process.


In [10]:
print(llm("Wer is Olaf Scholz? Bitte in Deutsch antworten"))


 Olaf Scholz ist ein deutscher Politiker. Er ist seit Dezember 2021 Bundeskanzler von Deutschland. Zuvor war er Finanzminister und Vizekanzler in der Regierung Merkel.


In [7]:
# Ollama embeddings
embeddings = OllamaEmbeddings(model="mistral")
# OpenAI embeddings
#embedding = OpenAIEmbeddings()

llm_open = Ollama(  model="mistral",
                    #model='Llama2',
                    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [8]:
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://newinmunich/langchain_course_indexers_retrievers already exists, loading from the storage


# Testing some text

In [18]:


# text to write to a local file
# taken from https://www.theverge.com/2023/3/14/23639313/google-ai-language-model-palm-api-challenge-openai
text = """Google opens up its AI language model PaLM to challenge OpenAI and GPT-3
Google is offering developers access to one of its most advanced AI language models: PaLM.
The search giant is launching an API for PaLM alongside a number of AI enterprise tools
it says will help businesses “generate text, images, code, videos, audio, and more from
simple natural language prompts.”

PaLM is a large language model, or LLM, similar to the GPT series created by OpenAI or
Meta’s LLaMA family of models. Google first announced PaLM in April 2022. Like other LLMs,
PaLM is a flexible system that can potentially carry out all sorts of text generation and
editing tasks. You could train PaLM to be a conversational chatbot like ChatGPT, for
example, or you could use it for tasks like summarizing text or even writing code.
(It’s similar to features Google also announced today for its Workspace apps like Google
Docs and Gmail.)
"""

# write text to local file
with open("my_file.txt", "w") as file:
    file.write(text)

# use TextLoader to load text from local file
loader = TextLoader("my_file.txt")
docs_from_file = loader.load()

print(len(docs_from_file))
# 1

1


In [19]:
from langchain.text_splitter import CharacterTextSplitter

# create a text splitter
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)

# split documents into chunks
docs = text_splitter.split_documents(docs_from_file)

print(len(docs))
# 2

Created a chunk of size 373, which is longer than the specified 200


2


In [25]:
db.add_documents(docs)

Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:16<00:00, 16.81s/it]

Dataset(path='hub://newinmunich/langchain_course_indexers_retrievers', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
 embedding  embedding  (2, 4096)  float32   None   
    id        text      (2, 1)      str     None   


['178ab390-c8ed-11ee-b55d-00155d0a12d7',
 '178ab430-c8ed-11ee-b55d-00155d0a12d7']

## Cleaning srt files

In [9]:
import re
def clean_srt_text(lines):  
    text = ''
    for line in lines:
        if re.search('^[0-9]+$', line) is None and re.search('^[0-9]{2}:[0-9]{2}:[0-9]{2}', line) is None and re.search('^$', line) is None:
            line = line.rstrip('\n')
            if line == "": 
                print("Empty line")
            else:
                text = text+ " " + line
    return text

In [10]:
def get_filenames_in_directory(directory):
    filenames = []
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            filenames.append(filename)
    return filenames

# Example usage:
source_directory = "./data/raw/"
target_directory = "./data/clean"


filenames = get_filenames_in_directory(source_directory)
print(filenames)

['11 - ND545 C1 L0 10 Lesson Recap - lang_en.srt', '5 - ND545 C1 L0 05 Cybersecurity And You Video - lang_en.srt', '3 - ND545 C1 L0 03 Lesson Outline Video - lang_en.srt', '2 - ND545 C1 L0 02 Course Outline Video - lang_en.srt', '1 - ND545 C1 L0 01 Meet Your Instructor Video - lang_en.srt', '7 - ND545 C1 L0 07 History Of Cybersecurity Video - lang_en.srt', '8 - ND545 C1 L0 08 Tools, Environment & Dependencies Pt 1 - lang_en.srt', '9 - ND545 C1 L0 08.1 Tools, Environment & Dependencies Pt 2 - lang_en.srt', '6 - Nd545-C1-L0-06-Business-Stakeholders-Video V2 - lang_en.srt', '4 - ND545 C1 L0 04 Introduction To Cybersecurity Video - lang_en.srt', '10 - ND545 C1 L0 09 Project- Cybersecurity Fundamentals Video - lang_en.srt']


In [11]:
old_extension = "srt"
new_extension = "txt"


In [12]:
def create_modified_filename(filename, directory, old_extension, new_extension):

    if filename.endswith(old_extension):
        old_path = os.path.join(directory, filename)
        new_filename = filename.rsplit(".", 1)[0] + "." + new_extension
        #new_path = os.path.join(directory, new_filename)
    return new_filename

In [13]:
for filename in filenames:
    print(create_modified_filename(filename, source_directory, old_extension, new_extension))

11 - ND545 C1 L0 10 Lesson Recap - lang_en.txt
5 - ND545 C1 L0 05 Cybersecurity And You Video - lang_en.txt
3 - ND545 C1 L0 03 Lesson Outline Video - lang_en.txt
2 - ND545 C1 L0 02 Course Outline Video - lang_en.txt
1 - ND545 C1 L0 01 Meet Your Instructor Video - lang_en.txt
7 - ND545 C1 L0 07 History Of Cybersecurity Video - lang_en.txt
8 - ND545 C1 L0 08 Tools, Environment & Dependencies Pt 1 - lang_en.txt
9 - ND545 C1 L0 08.1 Tools, Environment & Dependencies Pt 2 - lang_en.txt
6 - Nd545-C1-L0-06-Business-Stakeholders-Video V2 - lang_en.txt
4 - ND545 C1 L0 04 Introduction To Cybersecurity Video - lang_en.txt
10 - ND545 C1 L0 09 Project- Cybersecurity Fundamentals Video - lang_en.txt


In [18]:
def create_txt_from_srt(source_directory, target_directory, old_extention="srt", new_extension="txt"):
    filenames = get_filenames_in_directory(source_directory)
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
    for filename in filenames:
        
        with open(os.path.join(source_directory, filename), 'r', encoding='utf8') as f:
            lines = f.readlines()
        new_filename = create_modified_filename(filename, source_directory, old_extension, new_extension)
        clean_text = clean_srt_text(lines)

        new_file_path = os.path.join(target_directory, new_filename)
        with open(new_file_path, 'w') as f:
            f.write(clean_text)


In [19]:
create_txt_from_srt(source_directory, target_directory)

In [22]:
loader = DirectoryLoader(target_directory, glob="**/*.txt", show_progress=True)
docs = loader.load()

100%|██████████| 11/11 [00:00<00:00, 113.65it/s]


In [23]:
len(docs)

11